<a href="https://colab.research.google.com/github/zhangqianliZJU/Clustering/blob/master/TBM_smac_Gpyopt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !apt-get install build-essential swig
# !pip install smac
# !pip install gpyopt
# !mkdir results

In [5]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving rock mass properties.csv to rock mass properties.csv
User uploaded file "rock mass properties.csv" with length 5387 bytes


In [0]:
import logging
import numpy as np
import pandas as pd
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# Import ConfigSpace and different types of parameters
from smac.configspace import ConfigurationSpace
from ConfigSpace.hyperparameters import CategoricalHyperparameter, \
    UniformFloatHyperparameter, UniformIntegerHyperparameter
from ConfigSpace.conditions import InCondition

# Import SMAC-utilities
from smac.tae.execute_func import ExecuteTAFuncDict
from smac.scenario.scenario import Scenario
from smac.facade.smac_facade import SMAC
from smac.optimizer.acquisition import EI, PI, LCB
from smac.epm.rf_with_instances import RandomForestWithInstances
from smac.utils.util_funcs import get_types
from smac.utils.constants import MAXINT, N_TREES 
#import gpyopt
import GPy
import GPyOpt
#import random forest
from sklearn.ensemble import RandomForestRegressor
#import AdaBoostRegressor
from sklearn.ensemble import AdaBoostRegressor
#import a bagging regressor
from sklearn.ensemble import BaggingRegressor
#import keras
import keras
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras import losses
from keras.utils import plot_model#图像化输出网络结构
from keras.layers import LeakyReLU
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
#import normalization methods
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler,StandardScaler
# rockMassData = pd.read_csv('rock mass properties.csv',sep = ',')
# subRockMassData = rockMassData[['UCS','BTS','PSI','DPW','Alpha angle','Measured ROP']]
# subRockMassData['Alpha angle'] = np.sin(subRockMassData['Alpha angle'] * np.pi / 180)
# #Min-Max normalization
# maxData = subRockMassData.max()
# normData = subRockMassData/maxData
# totalFeatureLabels = np.array(normData)

# inputs = totalFeatureLabels[:,:-1]
# outputs = totalFeatureLabels[:,-1]
# trainInput,testInput,trainOutput,testOutput = train_test_split(inputs,outputs,train_size = 121,shuffle=True,random_state =123)
# # trainInput = inputs[:121,:]#The random split strategy performs better than the random split strategy
# # testInput = inputs[121:,:]
# # trainOutput = outputs[:121]
# # testOutput = outputs[121:]
# print(trainInput.shape,testInput.shape,trainOutput.shape,testOutput.shape)

def mae(trueOutput,predictOutput):
    mae = np.mean(np.abs(trueOutput-predictOutput))
    return mae
def mape(trueOutput,predictOutput):
    mape = np.mean(np.abs(trueOutput-predictOutput)/trueOutput)*100
    return mape
def coef(trueOutput,predictOutput):
    coef = np.corrcoef(trueOutput,predictOutput)
    return coef[0,1]
# def mse(trueOutput,predictOutput):
#     diff = (trueOutput-predictOutput) * maxData[5]
#     mse = np.mean(diff ** 2)
#     return mse

Using TensorFlow backend.


In [0]:
#optimize a svm model with SMAC
def svm_from_cfg(cfg):
#     print(cfg)
    cfg = {k : cfg[k] for k in cfg if cfg[k]}
#     print(cfg)
#     print(type(cfg['degree']))
    # We translate boolean values:
    cfg["shrinking"] = True if cfg["shrinking"] == "true" else False
    # And for gamma, we set it to a fixed value or to "auto" (if used)
    if "gamma" in cfg:
        cfg["gamma"] = cfg["gamma_value"] if cfg["gamma"] == "value" else "auto"
        cfg.pop("gamma_value", None)  # Remove "gamma_value"

    svr = svm.SVR(**cfg)
    svr.fit(trainInput,trainOutput)
    predictTestOutput = svr.predict(testInput)
#     testMAE = mae(testOutput,predictTestOutput)
#     testMAPE = mape(testOutput,predictTestOutput)
    testMSE = mse(testOutput,predictTestOutput)
#     print('Test MAE: %.5f, Test MAPE: %.5f'%(testMAE,testMAPE))
#     testCoef = coef(testOutput,predictTestOutput)
#     return testMAPE
    return testMSE
#     return -testCoef


#logger = logging.getLogger("SVMExample")
logging.basicConfig(level=logging.INFO)  # logging.DEBUG for debug output

# Build Configuration Space which defines all parameters and their ranges
cs = ConfigurationSpace()

# We define a few possible types of SVM-kernels and add them as "kernel" to our cs
kernel = CategoricalHyperparameter("kernel", ["linear", "rbf", "poly", "sigmoid"], default_value="poly")
cs.add_hyperparameter(kernel)

# There are some hyperparameters shared by all kernels
C = UniformFloatHyperparameter("C", 0, 1000.0, default_value=1)#default = 10
shrinking = CategoricalHyperparameter("shrinking", ["true", "false"], default_value="true")
cs.add_hyperparameters([C, shrinking])

#add epsilon
epsilon = UniformFloatHyperparameter("epsilon", 0, 1, default_value=0.1)     # Only used by kernel poly,default = 0.1

# Others are kernel-specific, so we can add conditions to limit the searchspace
degree = UniformIntegerHyperparameter("degree", 1, 5, default_value=3)     # Only used by kernel poly,default = 3
coef0 = UniformFloatHyperparameter("coef0", 0.0, 100.0, default_value=0.0)  # poly, sigmoid, default = 0.0
cs.add_hyperparameters([epsilon,degree, coef0])
use_degree = InCondition(child=degree, parent=kernel, values=["poly"])
use_coef0 = InCondition(child=coef0, parent=kernel, values=["poly", "sigmoid"])
cs.add_conditions([use_degree, use_coef0])

# This also works for parameters that are a mix of categorical and values from a range of numbers
# For example, gamma can be either "auto" or a fixed float
gamma = CategoricalHyperparameter("gamma", ["auto", "value"], default_value="auto")  # only rbf, poly, sigmoid
gamma_value = UniformFloatHyperparameter("gamma_value", 0, 20, default_value=10)# default = 3
cs.add_hyperparameters([gamma, gamma_value])
# We only activate gamma_value if gamma is set to "value"
cs.add_condition(InCondition(child=gamma_value, parent=gamma, values=["value"]))
# And again we can restrict the use of gamma in general to the choice of the kernel
cs.add_condition(InCondition(child=gamma, parent=kernel, values=["rbf", "poly", "sigmoid"]))


# Scenario object
scenario = Scenario({"run_obj": "quality",   # we optimize quality (alternatively runtime)
                     "runcount-limit": 150,  # maximum function evaluations
                     "cs": cs,               # configuration space
                     "deterministic": "true"
                     })
types, bounds = get_types(scenario.cs, scenario.feature_array)
rng =  np.random.RandomState(1)

model = RandomForestWithInstances(types=types,bounds=bounds,\
                                              instance_features=scenario.feature_array,\
                                              seed=rng.randint(MAXINT),\
                                              pca_components=scenario.PCA_DIM,\
                                              log_y=scenario.transform_y in ["LOG", "LOGS"],\
                                              num_trees=scenario.rf_num_trees,\
                                              do_bootstrapping=scenario.rf_do_bootstrapping,\
                                              ratio_features=scenario.rf_ratio_features,\
                                              min_samples_split=scenario.rf_min_samples_split,\
                                              min_samples_leaf=scenario.rf_min_samples_leaf,\
                                              max_depth=scenario.rf_max_depth)

# Example call of the function
# It returns: Status, Cost, Runtime, Additional Infos
def_value = svm_from_cfg(cs.get_default_configuration())
print("Default Value: %.2f" % (def_value))

# smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
#             tae_runner=svm_from_cfg,acquisition_function =EI(model=model))
smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
            tae_runner=svm_from_cfg,acquisition_function =PI(model=model))
incumbent = smac.optimize()
# print('The final incumbent is:',incumbent)
inc_value = svm_from_cfg(incumbent)

print("Optimized Value: %.2f" % (inc_value))

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2019-06-13_14:31:13_546540
INFO:smac.facade.smac_facade.SMAC:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic!


Default Value: 0.06


INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 0.0626
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 0.0626
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 0.0626
INFO:smac.intensification.intensification.Intensifier:Challenger (0.0526) is better than incumbent (0.0626) on 1 runs.
INFO:smac.intensification.intensification.Intensifier:Changes in incumbent:
INFO:smac.intensification.intensification.Intensifier:  C : 1.0 -> 250.11643606976952
INFO:smac.intensification.intensification.Intensifier:  coef0 : 0.0 -> 73.17740538538321
INFO:smac.intensification.intensification.Intensifier:  degree : 3 -> 1
INFO:smac.intensification.intensification.Intensifier:  epsilon : 0.1 -> 0.12018898146944634
INFO:smac.intensification.intensification.Intensifier:  gamma : 'auto' -> 'value'
INFO:smac.intensification.intensification.Intensifier:  gam

Optimized Value: 0.04


In [0]:
#Optimize a random forest model with smac
def rf_from_cfg(cfg):
#     print(cfg)
    cfg = {k : cfg[k] for k in cfg if cfg[k]}
#  

    rf = RandomForestRegressor(**cfg)
    rf.fit(trainInput,trainOutput)
    predictTestOutput = rf.predict(testInput)
#     testMAE = mae(testOutput,predictTestOutput)
#     testMAPE = mape(testOutput,predictTestOutput)
    testMSE = mse(testOutput,predictTestOutput)

#     print('Test MAE: %.5f, Test MAPE: %.5f'%(testMAE,testMAPE))
#     testCoef = coef(testOutput,predictTestOutput)
#     return testMAPE
    return testMSE
#     return -testCoef

#logger = logging.getLogger("SVMExample")
logging.basicConfig(level=logging.INFO)  # logging.DEBUG for debug output

# Build Configuration Space which defines all parameters and their ranges
cs = ConfigurationSpace()

numTrees = UniformIntegerHyperparameter("n_estimators", 5, 150, default_value=100)#number of estimators
maxDepth = UniformIntegerHyperparameter("max_depth", 2, 10, default_value=2)# The maximum depth of tree
min_samples_split = UniformIntegerHyperparameter("min_samples_split", 2, 6, default_value=2)#The min number of samples required to split an node
min_samples_leaf = UniformIntegerHyperparameter("min_samples_leaf", 1, 4, default_value=1)#The min number of samples at a leaf
maxFeatures = UniformIntegerHyperparameter("max_features", 3, 5, default_value=5)#The number of maximum features
cs.add_hyperparameters([numTrees, maxDepth,min_samples_split,min_samples_leaf,maxFeatures])


# Scenario object
scenario = Scenario({"run_obj": "quality",   # we optimize quality (alternatively runtime)
                     "runcount-limit": 150,  # maximum function evaluations
                     "cs": cs,               # configuration space
                     "deterministic": "true"
                     })
types, bounds = get_types(scenario.cs, scenario.feature_array)
rng =  np.random.RandomState(1)

model = RandomForestWithInstances(types=types,bounds=bounds,\
                                              instance_features=scenario.feature_array,\
                                              seed=rng.randint(MAXINT),\
                                              pca_components=scenario.PCA_DIM,\
                                              log_y=scenario.transform_y in ["LOG", "LOGS"],\
                                              num_trees=scenario.rf_num_trees,\
                                              do_bootstrapping=scenario.rf_do_bootstrapping,\
                                              ratio_features=scenario.rf_ratio_features,\
                                              min_samples_split=scenario.rf_min_samples_split,\
                                              min_samples_leaf=scenario.rf_min_samples_leaf,\
                                              max_depth=scenario.rf_max_depth)

# Example call of the function
# It returns: Status, Cost, Runtime, Additional Infos
def_value = rf_from_cfg(cs.get_default_configuration())
print("Default Value: %.2f" % (def_value))

# smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
#             tae_runner=svm_from_cfg,acquisition_function =EI(model=model))
smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
            tae_runner=rf_from_cfg,acquisition_function =PI(model=model))
incumbent = smac.optimize()
# print('The final incumbent is:',incumbent)
inc_value = rf_from_cfg(incumbent)

print("Optimized Value: %.2f" % (inc_value))

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2019-06-13_12:55:39_535310
INFO:smac.facade.smac_facade.SMAC:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic!


Default Value: 0.06


INFO:smac.intensification.intensification.Intensifier:Challenger (0.0469) is better than incumbent (0.0577) on 1 runs.
INFO:smac.intensification.intensification.Intensifier:Changes in incumbent:
INFO:smac.intensification.intensification.Intensifier:  max_depth : 2 -> 10
INFO:smac.intensification.intensification.Intensifier:  min_samples_leaf : 1 -> 2
INFO:smac.intensification.intensification.Intensifier:  min_samples_split : 2 -> 5
INFO:smac.intensification.intensification.Intensifier:  n_estimators : 100 -> 33
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 0.0469
INFO:smac.intensification.intensification.Intensifier:Challenger (0.0464) is better than incumbent (0.0469) on 1 runs.
INFO:smac.intensification.intensification.Intensifier:Changes in incumbent:
INFO:smac.intensification.intensification.Intensifier:  max_features : 5 -> 3
INFO:smac.intensification.intensification.Intensifier:  min_samples_leaf : 2 -> 4
INFO:smac.intensific

Optimized Value: 0.04


In [0]:
#Optimize an adaBoost model with smac
def adaBoost_from_cfg(cfg):
#     print(cfg)
    cfg = {k : cfg[k] for k in cfg if cfg[k]}
#  

    adaBoost = AdaBoostRegressor(**cfg)
    adaBoost.fit(trainInput,trainOutput)
    predictTestOutput = adaBoost.predict(testInput)
#     testMAE = mae(testOutput,predictTestOutput)
#     testMAPE = mape(testOutput,predictTestOutput)
    testMSE = mse(testOutput,predictTestOutput)

#     print('Test MAE: %.5f, Test MAPE: %.5f'%(testMAE,testMAPE))
#     testCoef = coef(testOutput,predictTestOutput)
#     return testMAPE
    return testMSE
#     return -testCoef

#logger = logging.getLogger("SVMExample")
logging.basicConfig(level=logging.INFO)  # logging.DEBUG for debug output

# Build Configuration Space which defines all parameters and their ranges
cs = ConfigurationSpace()

numEstimators = UniformIntegerHyperparameter("n_estimators", 5, 150, default_value=100)#number of estimators
learningRate = UniformFloatHyperparameter("learning_rate", 0.1, 10, default_value=1)# The learning rate
lossFunction = CategoricalHyperparameter("loss", ["linear", "square", "exponential"], default_value="linear")
cs.add_hyperparameters([numEstimators, learningRate,lossFunction])


# Scenario object
scenario = Scenario({"run_obj": "quality",   # we optimize quality (alternatively runtime)
                     "runcount-limit": 150,  # maximum function evaluations
                     "cs": cs,               # configuration space
                     "deterministic": "true"
                     })
types, bounds = get_types(scenario.cs, scenario.feature_array)
rng =  np.random.RandomState(1)

model = RandomForestWithInstances(types=types,bounds=bounds,\
                                              instance_features=scenario.feature_array,\
                                              seed=rng.randint(MAXINT),\
                                              pca_components=scenario.PCA_DIM,\
                                              log_y=scenario.transform_y in ["LOG", "LOGS"],\
                                              num_trees=scenario.rf_num_trees,\
                                              do_bootstrapping=scenario.rf_do_bootstrapping,\
                                              ratio_features=scenario.rf_ratio_features,\
                                              min_samples_split=scenario.rf_min_samples_split,\
                                              min_samples_leaf=scenario.rf_min_samples_leaf,\
                                              max_depth=scenario.rf_max_depth)

# Example call of the function
# It returns: Status, Cost, Runtime, Additional Infos
def_value = adaBoost_from_cfg(cs.get_default_configuration())
print("Default Value: %.2f" % (def_value))

# smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
#             tae_runner=svm_from_cfg,acquisition_function =EI(model=model))
smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
            tae_runner=adaBoost_from_cfg,acquisition_function =PI(model=model))
incumbent = smac.optimize()
# print('The final incumbent is:',incumbent)
inc_value = adaBoost_from_cfg(incumbent)
print("Optimized Value: %.2f" % (inc_value))

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2019-06-13_12:53:56_655427
INFO:smac.facade.smac_facade.SMAC:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic!


Default Value: 0.05


INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 0.0505
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 0.0505
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 0.0505
INFO:smac.intensification.intensification.Intensifier:Challenger (0.0469) is better than incumbent (0.0505) on 1 runs.
INFO:smac.intensification.intensification.Intensifier:Changes in incumbent:
INFO:smac.intensification.intensification.Intensifier:  learning_rate : 1.0 -> 0.7050512449655166
INFO:smac.intensification.intensification.Intensifier:  loss : 'linear' -> 'exponential'
INFO:smac.intensification.intensification.Intensifier:  n_estimators : 100 -> 101
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 0.0469
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 0.0

Optimized Value: 0.05


In [0]:
#Optimize an BaggingRegressor model with smac
def baggingRegressor_from_cfg(cfg):
#     print(cfg)
    cfg = {k : cfg[k] for k in cfg if cfg[k]}
    cfg['bootstrap'] = bool(cfg['bootstrap'])
    cfg['bootstrap_features'] = bool(cfg['bootstrap_features'])

    baggingRegressor = BaggingRegressor(**cfg)
    baggingRegressor.fit(trainInput,trainOutput)
    predictTestOutput = baggingRegressor.predict(testInput)
#     testMAE = mae(testOutput,predictTestOutput)
    testMAPE = mape(testOutput,predictTestOutput)
    testMSE = mse(testOutput,predictTestOutput)
#     print('Test MAE: %.5f, Test MAPE: %.5f'%(testMAE,testMAPE))
#     testCoef = coef(testOutput,predictTestOutput)
#     return testMAPE
    return testMSE
#     return -testCoef

#logger = logging.getLogger("SVMExample")
logging.basicConfig(level=logging.INFO)  # logging.DEBUG for debug output

# Build Configuration Space which defines all parameters and their ranges
cs = ConfigurationSpace()

numEstimators = UniformIntegerHyperparameter("n_estimators", 2, 100, default_value=100)#number of estimators
maxSamples = UniformFloatHyperparameter("max_samples", 0.5, 1.0, default_value=1.0)# The learning rate
maxFeatures = UniformIntegerHyperparameter("max_features", 2,5, default_value=5)
boostrap = CategoricalHyperparameter("bootstrap", ["true", "false"], default_value="true")
boostrap2 = CategoricalHyperparameter("bootstrap_features", ["true", "false"], default_value="false")


cs.add_hyperparameters([numEstimators,maxSamples,maxFeatures,boostrap,boostrap2])


# Scenario object
scenario = Scenario({"run_obj": "quality",   # we optimize quality (alternatively runtime)
                     "runcount-limit": 150,  # maximum function evaluations
                     "cs": cs,               # configuration space
                     "deterministic": "true"
                     })
types, bounds = get_types(scenario.cs, scenario.feature_array)
rng =  np.random.RandomState(1)

model = RandomForestWithInstances(types=types,bounds=bounds,\
                                              instance_features=scenario.feature_array,\
                                              seed=rng.randint(MAXINT),\
                                              pca_components=scenario.PCA_DIM,\
                                              log_y=scenario.transform_y in ["LOG", "LOGS"],\
                                              num_trees=scenario.rf_num_trees,\
                                              do_bootstrapping=scenario.rf_do_bootstrapping,\
                                              ratio_features=scenario.rf_ratio_features,\
                                              min_samples_split=scenario.rf_min_samples_split,\
                                              min_samples_leaf=scenario.rf_min_samples_leaf,\
                                              max_depth=scenario.rf_max_depth)

# Example call of the function
# It returns: Status, Cost, Runtime, Additional Infos
def_value = baggingRegressor_from_cfg(cs.get_default_configuration())
print("Default Value: %.2f" % (def_value))

# smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
#             tae_runner=svm_from_cfg,acquisition_function =EI(model=model))
smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
            tae_runner=baggingRegressor_from_cfg,acquisition_function =EI(model=model))
incumbent = smac.optimize()
# print('The final incumbent is:',incumbent)
inc_value = baggingRegressor_from_cfg(incumbent)
print("Optimized Value: %.2f" % (inc_value))

NameError: ignored

In [0]:
# A self implemented AutoML framework
# Feature engineering,machine learning model selection, hyperparameter optimization
rockMassData = pd.read_csv('rock mass properties.csv',sep = ',')
subRockMassData = rockMassData[['UCS','BTS','PSI','DPW','Alpha angle','Measured ROP']]
subRockMassData['Alpha angle'] = np.sin(subRockMassData['Alpha angle'] * np.pi / 180)
trainInput = None
trainOutput = None
testInput = None
testOutput = None
#Normalization methods selection
def getNormalizator(norm_method):
    normMethod = None
    if norm_method == 'standard':
        normMethod = StandardScaler
    if norm_method == 'min-max':
        normMethod = MinMaxScaler
    if norm_method == 'max-abs':
        normMethod = MaxAbsScaler
    return normMethod
        
    
#Model selection
def getRegressor(modelName):
    regressor = None
    if modelName == 'randomForest':
        regressor = RandomForestRegressor
    if modelName == 'adaBoost':
        regressor = AdaBoostRegressor
    return regressor        
def regressor_from_cfg(cfg):
    cfg = {k : cfg[k] for k in cfg if cfg[k]}
    #Generate model and pop the item
    norm_fuck = cfg['norm-method']
    cfg.pop('norm-method')
    rockMassData = pd.read_csv('rock mass properties.csv',sep = ',')
    subRockMassData = rockMassData[['UCS','BTS','PSI','DPW','Alpha angle','Measured ROP']]
    subRockMassData['Alpha angle'] = np.sin(subRockMassData['Alpha angle'] * np.pi / 180)
    totalFeatureLabels = np.array(subRockMassData)
    norm_constructor = getNormalizator(norm_fuck)
    normModel = norm_constructor()
    normModel.fit_transform(totalFeatureLabels)    
    inputs = totalFeatureLabels[:,:-1]
    outputs = totalFeatureLabels[:,-1]
    trainInput,testInput,trainOutput,testOutput = train_test_split(inputs,outputs,train_size = 121,shuffle=True,random_state =123)
    
    modelName = cfg['modelName']
    regressor = getRegressor(modelName)#Constructor for the specified regressor model    
    cfg.pop('modelName')
    #Create the regressor model
    model = regressor(**cfg)

    model.fit(trainInput,trainOutput)
    predictTestOutput = model.predict(testInput)
#     testMAE = mae(testOutput,predictTestOutput)
    testMAPE = mape(testOutput,predictTestOutput)
#     testMSE = mse(testOutput,predictTestOutput)
#     print('Test MAE: %.5f, Test MAPE: %.5f'%(testMAE,testMAPE))
#     testCoef = coef(testOutput,predictTestOutput)
#     return testMSE
    return testMAPE
#     return -testCoef

#logger = logging.getLogger("SVMExample")
logging.basicConfig(level=logging.INFO)  # logging.DEBUG for debug output
# Build Configuration Space which defines all parameters and their ranges
cs = ConfigurationSpace()
#Add the preprocessing option for the AutoML framework TO-DO
# Pattern of normalization, The 151 data should be splited sequentially, the former 121 for training
#The latter 30 for testing

# data prerpocessing
norm_methods = CategoricalHyperparameter("norm-method", ["standard", "min-max","max-abs"], default_value="min-max")
# model selection and optimizatio of hyperparameters
models = CategoricalHyperparameter("modelName", ["randomForest", "adaBoost"], default_value="adaBoost")
cs.add_hyperparameters([norm_methods,models])
#hyperParameters for svr

#hyperParameters for random forest
numEstimators = UniformIntegerHyperparameter("n_estimators", 5, 150, default_value=100)#number of estimators
maxDepth = UniformIntegerHyperparameter("max_depth", 2, 10, default_value=2)# The maximum depth of tree
min_samples_split = UniformIntegerHyperparameter("min_samples_split", 2, 6, default_value=2)#The min number of samples required to split an node
min_samples_leaf = UniformIntegerHyperparameter("min_samples_leaf", 1, 4, default_value=1)#The min number of samples at a leaf
maxFeatures = UniformIntegerHyperparameter("max_features", 3, 5, default_value=5)#The number of maximum features
cs.add_hyperparameters([numEstimators,maxDepth,min_samples_split,min_samples_leaf,maxFeatures])

use_maxDepth = InCondition(child=maxDepth, parent=models, values=["randomForest"])
use_min_samples_split = InCondition(child=min_samples_split, parent=models, values=["randomForest"])
use_min_samples_leaf = InCondition(child=min_samples_leaf, parent=models, values=["randomForest"])
use_maxFeatures = InCondition(child=maxFeatures, parent=models, values=["randomForest"])
cs.add_conditions([use_maxDepth,use_min_samples_split,use_min_samples_leaf,use_maxFeatures])

#hyperParameters for adaBoost
# numEstimators = UniformIntegerHyperparameter("n_estimators", 5, 150, default_value=100)#number of estimators
learningRate = UniformFloatHyperparameter("learning_rate", 0.1, 10, default_value=1)# The learning rate
lossFunction = CategoricalHyperparameter("loss", ["linear", "square", "exponential"], default_value="linear")
cs.add_hyperparameters([learningRate,lossFunction])

use_learningRate = InCondition(child=learningRate, parent=models, values=["adaBoost"])
use_lossFunction = InCondition(child=lossFunction, parent=models, values=["adaBoost"])

cs.add_conditions([use_learningRate, use_lossFunction])
# Scenario object
scenario = Scenario({"run_obj": "quality",   # we optimize quality (alternatively runtime)
                     "runcount-limit": 300,  # maximum function evaluations
                     "cs": cs,               # configuration space
                     "deterministic": "true"
                     })
types, bounds = get_types(scenario.cs, scenario.feature_array)
rng =  np.random.RandomState(1)

model = RandomForestWithInstances(types=types,bounds=bounds,\
                                              instance_features=scenario.feature_array,\
                                              seed=rng.randint(MAXINT),\
                                              pca_components=scenario.PCA_DIM,\
                                              log_y=scenario.transform_y in ["LOG", "LOGS"],\
                                              num_trees=scenario.rf_num_trees,\
                                              do_bootstrapping=scenario.rf_do_bootstrapping,\
                                              ratio_features=scenario.rf_ratio_features,\
                                              min_samples_split=scenario.rf_min_samples_split,\
                                              min_samples_leaf=scenario.rf_min_samples_leaf,\
                                              max_depth=scenario.rf_max_depth)

# Example call of the function
# It returns: Status, Cost, Runtime, Additional Infos
def_value = regressor_from_cfg(cs.get_default_configuration())
print("Default Value: %.2f" % (def_value))

# smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
#             tae_runner=svm_from_cfg,acquisition_function =EI(model=model))
smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
            tae_runner=regressor_from_cfg,acquisition_function =EI(model=model))
incumbent = smac.optimize()
# print('The final incumbent is:',incumbent)
inc_value = regressor_from_cfg(incumbent)
print("Optimized Value: %.2f" % (inc_value))

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2019-06-11_06:16:11_813173
INFO:smac.facade.smac_facade.SMAC:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic!


Default Value: 10.09


INFO:smac.intensification.intensification.Intensifier:Challenger (9.6565) is better than incumbent (10.5126) on 1 runs.
INFO:smac.intensification.intensification.Intensifier:Changes in incumbent:
INFO:smac.intensification.intensification.Intensifier:  max_depth : None -> 7
INFO:smac.intensification.intensification.Intensifier:  max_features : None -> 3
INFO:smac.intensification.intensification.Intensifier:  min_samples_leaf : None -> 1
INFO:smac.intensification.intensification.Intensifier:  min_samples_split : None -> 3
INFO:smac.intensification.intensification.Intensifier:  modelName : 'adaBoost' -> 'randomForest'
INFO:smac.intensification.intensification.Intensifier:  n_estimators : 100 -> 46
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 9.6565
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 9.6565
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of 

Optimized Value: 10.11


In [0]:
# A self implemented AutoML framework with three regressors:svr,random forest,adaboost
# Feature engineering,machine learning model selection, hyperparameter optimization
# def mse(trueOutput,predictOutput):
#     diff = (trueOutput-predictOutput)
#     mse = np.mean(diff ** 2)
#     return mse
#Normalization methods selection
def getNormalizator(norm_method):
    normMethod = None
    if norm_method == 'standard':
        normMethod = StandardScaler
    if norm_method == 'min-max':
        normMethod = MinMaxScaler
    if norm_method == 'max-abs':
        normMethod = MaxAbsScaler
    return normMethod

#Model selection
def getRegressor(modelName):
    regressor = None
    if modelName == 'randomForest':
        regressor = RandomForestRegressor
    if modelName == 'adaBoost':
        regressor = AdaBoostRegressor
    if modelName == 'svr':
        regressor = svm.SVR
    return regressor        
def regressor_from_cfg(cfg):
    cfg = {k : cfg[k] for k in cfg if cfg[k]}
    #Generate model and pop the item
    norm_fuck = cfg['norm-method']
    cfg.pop('norm-method')
    rockMassData = pd.read_csv('rock mass properties.csv',sep = ',')
    subRockMassData = rockMassData[['UCS','BTS','PSI','DPW','Alpha angle','Measured ROP']]
    subRockMassData['Alpha angle'] = np.sin(subRockMassData['Alpha angle'] * np.pi / 180)
    totalFeatureLabels = np.array(subRockMassData)
    norm_constructor = getNormalizator(norm_fuck)
    normModel = norm_constructor()
    totalFeatureLabels_scaled = normModel.fit_transform(totalFeatureLabels)    
    inputs = totalFeatureLabels_scaled[:,:-1]
    outputs = totalFeatureLabels_scaled[:,-1]
#     trainInput,testInput,trainOutput,_ = train_test_split(inputs,outputs,train_size = 121,shuffle=True,random_state =123)
    trainInput = inputs[:121,:]#The random split strategy performs better than the random split strategy
    testInput = inputs[121:,:]
    trainOutput = outputs[:121]
    testOutput = outputs[121:]    
    modelName = cfg['modelName']
    if modelName == 'svr':
        print(cfg)
        cfg["shrinking"] = True if cfg["shrinking"] == "true" else False
        if "gamma" in cfg:
            cfg["gamma"] = cfg["gamma_value"] if cfg["gamma"] == "value" else "auto"
            cfg.pop("gamma_value", None)  # Remove "gamma_value"
    regressor = getRegressor(modelName)#Constructor for the specified regressor model    
    cfg.pop('modelName')
    #Create the regressor model
    model = regressor(**cfg)

    model.fit(trainInput,trainOutput)
    predictTestOutput = model.predict(testInput)
    restored = np.zeros((30,6))
    restored[:,:5] = testInput
    restored[:,5] = predictTestOutput
    restored_predict = normModel.inverse_transform(restored)
	
#     testMAE = mae(testOutput,predictTestOutput)
#     testMAPE = mape(testOutput,predictTestOutput)
    testMSE = mse(totalFeatureLabels[121:,5],restored_predict[:,5])
#     print('Test MAE: %.5f, Test MAPE: %.5f'%(testMAE,testMAPE))
#     testCoef = coef(testOutput,predictTestOutput)
    return testMSE
#    return testMAPE
#     return -testCoef

#logger = logging.getLogger("SVMExample")
logging.basicConfig(level=logging.INFO)  # logging.DEBUG for debug output
# Build Configuration Space which defines all parameters and their ranges
cs = ConfigurationSpace()
#Add the preprocessing option for the AutoML framework TO-DO
# Pattern of normalization, The 151 data should be splited sequentially, the former 121 for training
#The latter 30 for testing

# data prerpocessing
norm_methods = CategoricalHyperparameter("norm-method", ["standard", "min-max","max-abs"], default_value="min-max")
# model selection and optimizatio of hyperparameters
models = CategoricalHyperparameter("modelName", ["randomForest", "adaBoost","svr"], default_value="svr")
cs.add_hyperparameters([norm_methods,models])

#hyperParameters for svr: C, kernel, shrinking, degree <- kernel:poly, coef0 <- poly, sigmoid, gamma <-rbf, "poly, sigmoid
# We define a few possible types of SVM-kernels and add them as "kernel" to our cs
kernel = CategoricalHyperparameter("kernel", ["linear", "rbf", "poly", "sigmoid"], default_value="poly")
C = UniformFloatHyperparameter("C", 0, 1000.0, default_value=1)#default = 10
shrinking = CategoricalHyperparameter("shrinking", ["true", "false"], default_value="true")
cs.add_hyperparameters([kernel,C, shrinking])
use_C = InCondition(child=C,parent=models,values=["svr"])
use_kernel = InCondition(child=kernel,parent=models,values=["svr"])
use_shrinking = InCondition(child=shrinking,parent=models,values=["svr"])
cs.add_conditions([use_kernel,use_C,use_shrinking])

# Others are kernel-specific, so we can add conditions to limit the searchspace
degree = UniformIntegerHyperparameter("degree", 1, 5, default_value=3)     # Only used by kernel poly,default = 3
coef0 = UniformFloatHyperparameter("coef0", 0.0, 100.0, default_value=0.0)  # poly, sigmoid, default = 0.0
cs.add_hyperparameters([degree, coef0])

use_degree = InCondition(child=degree, parent=kernel, values=["poly"])
use_coef0 = InCondition(child=coef0, parent=kernel, values=["poly", "sigmoid"])

# use_degree = InCondition(child=degree, parent=models, values=["svr"])
# use_coef0 = InCondition(child=coef0, parent=models, values=["svr"])
cs.add_conditions([use_degree, use_coef0])

# This also works for parameters that are a mix of categorical and values from a range of numbers
# For example, gamma can be either "auto" or a fixed float
gamma = CategoricalHyperparameter("gamma", ["auto", "value"], default_value="auto")  # only rbf, poly, sigmoid
gamma_value = UniformFloatHyperparameter("gamma_value", 0, 20, default_value=10)# default = 3
cs.add_hyperparameters([gamma, gamma_value])
# We only activate gamma_value if gamma is set to "value"
cs.add_condition(InCondition(child=gamma_value, parent=gamma, values=["value"]))
# And again we can restrict the use of gamma in general to the choice of the kernel
cs.add_condition(InCondition(child=gamma, parent=kernel, values=["rbf", "poly", "sigmoid"]))

#hyperParameters for random forest
numEstimators = UniformIntegerHyperparameter("n_estimators", 5, 150, default_value=100)#number of estimators

maxDepth = UniformIntegerHyperparameter("max_depth", 2, 10, default_value=2)# The maximum depth of tree
min_samples_split = UniformIntegerHyperparameter("min_samples_split", 2, 6, default_value=2)#The min number of samples required to split an node
min_samples_leaf = UniformIntegerHyperparameter("min_samples_leaf", 1, 4, default_value=1)#The min number of samples at a leaf
maxFeatures = UniformIntegerHyperparameter("max_features", 3, 5, default_value=5)#The number of maximum features
cs.add_hyperparameters([numEstimators,maxDepth,min_samples_split,min_samples_leaf,maxFeatures])

use_maxDepth = InCondition(child=maxDepth, parent=models, values=["randomForest"])
use_min_samples_split = InCondition(child=min_samples_split, parent=models, values=["randomForest"])
use_min_samples_leaf = InCondition(child=min_samples_leaf, parent=models, values=["randomForest"])
use_maxFeatures = InCondition(child=maxFeatures, parent=models, values=["randomForest"])
cs.add_conditions([use_maxDepth,use_min_samples_split,use_min_samples_leaf,use_maxFeatures])

#hyperParameters for adaBoost
# numEstimators = UniformIntegerHyperparameter("n_estimators", 5, 150, default_value=100)#number of estimators
learningRate = UniformFloatHyperparameter("learning_rate", 0.1, 10, default_value=1)# The learning rate
lossFunction = CategoricalHyperparameter("loss", ["linear", "square", "exponential"], default_value="linear")
cs.add_hyperparameters([learningRate,lossFunction])

use_learningRate = InCondition(child=learningRate, parent=models, values=["adaBoost"])
use_lossFunction = InCondition(child=lossFunction, parent=models, values=["adaBoost"])
use_numEstimators = InCondition(child=numEstimators,parent=models,values=["randomForest","adaBoost"])
cs.add_conditions([use_numEstimators,use_learningRate, use_lossFunction])
# Scenario object
scenario = Scenario({"run_obj": "quality",   # we optimize quality (alternatively runtime)
                     "runcount-limit": 300,  # maximum function evaluations
                     "cs": cs,               # configuration space
                     "deterministic": "true"})
types, bounds = get_types(scenario.cs, scenario.feature_array)
rng =  np.random.RandomState(1)

model = RandomForestWithInstances(types=types,bounds=bounds,\
                                              instance_features=scenario.feature_array,\
                                              seed=rng.randint(MAXINT),\
                                              pca_components=scenario.PCA_DIM,\
                                              log_y=scenario.transform_y in ["LOG", "LOGS"],\
                                              num_trees=scenario.rf_num_trees,\
                                              do_bootstrapping=scenario.rf_do_bootstrapping,\
                                              ratio_features=scenario.rf_ratio_features,\
                                              min_samples_split=scenario.rf_min_samples_split,\
                                              min_samples_leaf=scenario.rf_min_samples_leaf,\
                                              max_depth=scenario.rf_max_depth)

# Example call of the function
# It returns: Status, Cost, Runtime, Additional Infos
def_value = regressor_from_cfg(cs.get_default_configuration())
print("Default Value: %.2f" % (def_value))

# smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
#             tae_runner=svm_from_cfg,acquisition_function =EI(model=model))
smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
            tae_runner=regressor_from_cfg,acquisition_function =EI(model=model))
incumbent = smac.optimize()
# print('The final incumbent is:',incumbent)
inc_value = regressor_from_cfg(incumbent)
print("Optimized Value: %.2f" % (inc_value))

In [0]:
#The gamma is always set to value, not "auto"
# A self implemented AutoML framework with three regressors:svr,random forest,adaboost
# Feature engineering,machine learning model selection, hyperparameter optimization
logging.basicConfig(level=logging.INFO)  # logging.DEBUG for debug output
rockMassData = pd.read_csv('rock mass properties.csv',sep = ',')
subRockMassData = rockMassData[['UCS','BTS','PSI','DPW','Alpha angle','Measured ROP']]
subRockMassData['Alpha angle'] = np.sin(subRockMassData['Alpha angle'] * np.pi / 180)
totalFeatureLabels = np.array(subRockMassData)
def mse(trueOutput,predictOutput):
    diff = (trueOutput-predictOutput)
    mse = np.mean(diff ** 2)
    return mse
#Normalization methods selection
def getNormalizator(norm_method):
    normMethod = None
    if norm_method == 'standard':
        normMethod = StandardScaler
    if norm_method == 'min-max':
        normMethod = MinMaxScaler
    if norm_method == 'max-abs':
        normMethod = MaxAbsScaler
    return normMethod

#Model selection
def getRegressor(modelName):
    regressor = None
    if modelName == 'randomForest':
        regressor = RandomForestRegressor
    if modelName == 'adaBoost':
        regressor = AdaBoostRegressor
    if modelName == 'svr':
        regressor = svm.SVR
    return regressor        
def regressor_from_cfg(cfg):
    cfg = {k : cfg[k] for k in cfg if cfg[k]}
    #Generate model and pop the item
    norm_fuck = cfg['norm-method']
    cfg.pop('norm-method')
#     rockMassData = pd.read_csv('rock mass properties.csv',sep = ',')
#     subRockMassData = rockMassData[['UCS','BTS','PSI','DPW','Alpha angle','Measured ROP']]
#     subRockMassData['Alpha angle'] = np.sin(subRockMassData['Alpha angle'] * np.pi / 180)
#     totalFeatureLabels = np.array(subRockMassData)
    norm_constructor = getNormalizator(norm_fuck)
    normModel = norm_constructor()
    totalFeatureLabels_scaled = normModel.fit_transform(totalFeatureLabels)    
    inputs = totalFeatureLabels_scaled[:,:-1]
    outputs = totalFeatureLabels_scaled[:,-1]
    trainInput,testInput,trainOutput,_ = train_test_split(inputs,outputs,train_size = 121,shuffle=True,random_state =123)
#     trainInput = inputs[:121,:]#The random split strategy performs better than the random split strategy
#     testInput = inputs[121:,:]
#     trainOutput = outputs[:121]
#     testOutput = outputs[121:]    
    modelName = cfg['modelName']
    if modelName == 'svr':
        print(cfg)
        cfg["shrinking"] = True if cfg["shrinking"] == "true" else False
        if "gamma" in cfg:
            cfg["gamma"] = cfg["gamma_value"] if cfg["gamma"] == "value" else "auto"
            cfg.pop("gamma_value", None)  # Remove "gamma_value"
    regressor = getRegressor(modelName)#Constructor for the specified regressor model    
    cfg.pop('modelName')
    #Create the regressor model
    model = regressor(**cfg)

    model.fit(trainInput,trainOutput)
    predictTestOutput = model.predict(testInput)
    restored = np.zeros((30,6))
    restored[:,:5] = testInput
    restored[:,5] = predictTestOutput
    restored_predict = normModel.inverse_transform(restored)
	
#     testMAE = mae(testOutput,predictTestOutput)
#     testMAPE = mape(testOutput,predictTestOutput)
    testMSE = mse(totalFeatureLabels[121:,5],restored_predict[:,5])
#     print('Test MAE: %.5f, Test MAPE: %.5f'%(testMAE,testMAPE))
#     testCoef = coef(testOutput,predictTestOutput)
    print
    return testMSE
#    return testMAPE
#     return -testCoef

#logger = logging.getLogger("SVMExample") 
logging.basicConfig(level=logging.INFO)  # logging.DEBUG for debug output
# Build Configuration Space which defines all parameters and their ranges
cs = ConfigurationSpace()
#Add the preprocessing option for the AutoML framework TO-DO
# Pattern of normalization, The 151 data should be splited sequentially, the former 121 for training
#The latter 30 for testing

# data prerpocessing
norm_methods = CategoricalHyperparameter("norm-method", ["standard", "min-max","max-abs"], default_value="min-max")
# model selection and optimizatio of hyperparameters
models = CategoricalHyperparameter("modelName", ["randomForest", "adaBoost","svr"], default_value="svr")
cs.add_hyperparameters([norm_methods,models])

#hyperParameters for svr: C, kernel, shrinking, degree <- kernel:poly, coef0 <- poly, sigmoid, gamma <-rbf, "poly, sigmoid
# We define a few possible types of SVM-kernels and add them as "kernel" to our cs
kernel = CategoricalHyperparameter("kernel", ["linear", "rbf", "poly", "sigmoid"], default_value="poly")
C = UniformFloatHyperparameter("C", 500, 1000.0, default_value=600)#default = 10
shrinking = CategoricalHyperparameter("shrinking", ["true", "false"], default_value="true")
cs.add_hyperparameters([kernel,C, shrinking])
use_C = InCondition(child=C,parent=models,values=["svr"])
use_kernel = InCondition(child=kernel,parent=models,values=["svr"])
use_shrinking = InCondition(child=shrinking,parent=models,values=["svr"])
cs.add_conditions([use_kernel,use_C,use_shrinking])

# Others are kernel-specific, so we can add conditions to limit the searchspace
degree = UniformIntegerHyperparameter("degree", 1, 5, default_value=3)     # Only used by kernel poly,default = 3
coef0 = UniformFloatHyperparameter("coef0", 0.0, 10.0, default_value=0.0)  # poly, sigmoid, default = 0.0
cs.add_hyperparameters([degree, coef0])

use_degree = InCondition(child=degree, parent=kernel, values=["poly"])
use_coef0 = InCondition(child=coef0, parent=kernel, values=["poly", "sigmoid"])

# use_degree = InCondition(child=degree, parent=models, values=["svr"])
# use_coef0 = InCondition(child=coef0, parent=models, values=["svr"])
cs.add_conditions([use_degree, use_coef0])

# This also works for parameters that are a mix of categorical and values from a range of numbers
# For example, gamma can be either "auto" or a fixed float
gamma = CategoricalHyperparameter("gamma", ["auto", "value"], default_value="auto")  # only rbf, poly, sigmoid
gamma_value = UniformFloatHyperparameter("gamma_value", 0, 20, default_value=3)# default = 3
cs.add_hyperparameters([gamma,gamma_value])
# We only activate gamma_value if gamma is set to "value"
cs.add_condition(InCondition(child=gamma_value, parent=gamma, values=["value"]))
# And again we can restrict the use of gamma in general to the choice of the kernel
cs.add_condition(InCondition(child=gamma, parent=kernel, values=["rbf", "poly", "sigmoid"]))

#hyperParameters for random forest
numEstimators = UniformIntegerHyperparameter("n_estimators", 5, 150, default_value=100)#number of estimators

maxDepth = UniformIntegerHyperparameter("max_depth", 2, 10, default_value=2)# The maximum depth of tree
min_samples_split = UniformIntegerHyperparameter("min_samples_split", 2, 6, default_value=2)#The min number of samples required to split an node
min_samples_leaf = UniformIntegerHyperparameter("min_samples_leaf", 1, 4, default_value=1)#The min number of samples at a leaf
maxFeatures = UniformIntegerHyperparameter("max_features", 3, 5, default_value=5)#The number of maximum features
cs.add_hyperparameters([numEstimators,maxDepth,min_samples_split,min_samples_leaf,maxFeatures])

use_maxDepth = InCondition(child=maxDepth, parent=models, values=["randomForest"])
use_min_samples_split = InCondition(child=min_samples_split, parent=models, values=["randomForest"])
use_min_samples_leaf = InCondition(child=min_samples_leaf, parent=models, values=["randomForest"])
use_maxFeatures = InCondition(child=maxFeatures, parent=models, values=["randomForest"])
cs.add_conditions([use_maxDepth,use_min_samples_split,use_min_samples_leaf,use_maxFeatures])

#hyperParameters for adaBoost
# numEstimators = UniformIntegerHyperparameter("n_estimators", 5, 150, default_value=100)#number of estimators
learningRate = UniformFloatHyperparameter("learning_rate", 0.1, 10, default_value=1)# The learning rate
lossFunction = CategoricalHyperparameter("loss", ["linear", "square", "exponential"], default_value="linear")
cs.add_hyperparameters([learningRate,lossFunction])

use_learningRate = InCondition(child=learningRate, parent=models, values=["adaBoost"])
use_lossFunction = InCondition(child=lossFunction, parent=models, values=["adaBoost"])
use_numEstimators = InCondition(child=numEstimators,parent=models,values=["randomForest","adaBoost"])
cs.add_conditions([use_numEstimators,use_learningRate, use_lossFunction])
# Scenario object
scenario = Scenario({"run_obj": "quality",   # we optimize quality (alternatively runtime)
                     "runcount-limit": 150,  # maximum function evaluations
                     "cs": cs,               # configuration space
                     "deterministic": "true"})
types, bounds = get_types(scenario.cs, scenario.feature_array)
rng =  np.random.RandomState(1)

model = RandomForestWithInstances(types=types,bounds=bounds,\
                                              instance_features=scenario.feature_array,\
                                              seed=rng.randint(MAXINT),\
                                              pca_components=scenario.PCA_DIM,\
                                              log_y=scenario.transform_y in ["LOG", "LOGS"],\
                                              num_trees=scenario.rf_num_trees,\
                                              do_bootstrapping=scenario.rf_do_bootstrapping,\
                                              ratio_features=scenario.rf_ratio_features,\
                                              min_samples_split=scenario.rf_min_samples_split,\
                                              min_samples_leaf=scenario.rf_min_samples_leaf,\
                                              max_depth=scenario.rf_max_depth)

# Example call of the function
# It returns: Status, Cost, Runtime, Additional Infos
def_value = regressor_from_cfg(cs.get_default_configuration())
print("Default Value: %.6f" % (def_value))

# smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
#             tae_runner=svm_from_cfg,acquisition_function =EI(model=model))
smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
            tae_runner=regressor_from_cfg,acquisition_function =LCB(model=model))
incumbent = smac.optimize()
# print('The final incumbent is:',incumbent)
inc_value = regressor_from_cfg(incumbent)
print("Optimized Value: %.6f" % (inc_value))

{'modelName': 'svr', 'C': 600.0, 'kernel': 'poly', 'shrinking': 'true', 'degree': 3, 'gamma': 'auto'}
Default Value: 0.187447
{'modelName': 'svr', 'C': 600.0, 'kernel': 'poly', 'shrinking': 'true', 'degree': 3, 'gamma': 'auto'}
{'modelName': 'svr', 'C': 795.3293265801383, 'kernel': 'rbf', 'shrinking': 'false', 'gamma': 'auto'}


KeyboardInterrupt: ignored

In [0]:
# random forest with gpyopt，poorer performance than smac
def rf_gopt_obj(x):     
    hyperParameters = {}
    hyperParameters['n_estimators'] = int(x[0,0])
    hyperParameters['max_depth'] = int(x[0,1])
    hyperParameters['min_samples_split'] = int(x[0,2])
    hyperParameters['min_samples_leaf'] = int(x[0,3])
    hyperParameters['max_features'] = int(x[0,4])
#     print(hyperParameters)
    rf = RandomForestRegressor(**hyperParameters)    
    
    rf.fit(trainInput,trainOutput)
    predictTestOutput = rf.predict(testInput)
    testMAE = mae(testOutput,predictTestOutput)
    testMAPE = mape(testOutput,predictTestOutput)
#     print('Test MAE: %.5f, Test MAPE: %.5f'%(testMAE,testMAPE))
    return testMAPE


# numTrees = UniformIntegerHyperparameter("n_estimators", 5, 150, default_value=100)#number of estimators
# maxDepth = UniformIntegerHyperparameter("max_depth", 2, 10, default_value=2)# The maximum depth of tree
# min_samples_split = UniformIntegerHyperparameter("min_samples_split", 2, 6, default_value=2)#The min number of samples required to split an node
# min_samples_leaf = UniformIntegerHyperparameter("min_samples_leaf", 1, 4, default_value=1)#The min number of samples at a leaf
# maxFeatures = UniformIntegerHyperparameter("max_features", 3, 5, default_value=5)#The number of maximum features

numTree_domain_list = [i for i in range(2,200)]
maxDepth_domain_list = [i for i in range(2,11)]
minSplit_domain_list = [i for i in range(2,10)]
minLeaf_domain_list = [i for i in range(1,5)]
maxFeatures_domain_list = [i for i in range(2,6)]

bounds = [{'name': 'numTrees', 'type': 'discrete', 'domain': tuple(numTree_domain_list)},
               {'name': 'maxDepth','type': 'discrete', 'domain': tuple(maxDepth_domain_list)},
               {'name': 'min_samples_split',  'type': 'discrete', 'domain':tuple(minSplit_domain_list)},
               {'name':'min_samples_leaf','type':'discrete','domain':tuple(minLeaf_domain_list)},
               {'name': 'maxFeatures','type': 'discrete', 'domain': tuple(maxFeatures_domain_list)}] 

acquisition_funcs = ['EI', 'MPI', 'LCB']#Each acquisition functions should be tested
# store the convergence history of each acquisition function in a dictionary
all_convergence_hist = {}
# seed(12345)
for acqui_func in acquisition_funcs:
    print(acqui_func)   
#     myBopt = GPyOpt.methods.BayesianOptimization(f=rf_gopt_obj,domain=bounds,initial_design_numdata=5,\
#                                                  acquisition_type=acqui_func,verbosity=False,evaluator_type='local_penalization',batch_size=8)
    myBopt = GPyOpt.methods.BayesianOptimization(f=rf_gopt_obj,domain=bounds,initial_design_numdata=5,\
                                                 acquisition_type=acqui_func,verbosity=False)
    myBopt.run_optimization(100)
    myBopt.plot_convergence()
    print(myBopt.x_opt)
    print(myBopt.fx_opt)
    evals = myBopt.get_evaluations()
    dv_hist = evals[0]
    objective_hist = evals[1]
    epochs = objective_hist.shape[0]
    converge_hist = np.empty((epochs,))
    minObj = objective_hist[0,0]
    for index in range(epochs):
        histIter = objective_hist[index,0]
        if minObj >= histIter:
            minObj = histIter
        converge_hist[index] = minObj
    all_convergence_hist[acqui_func] = converge_hist
# plot the convergence trace of each acquisition function
plt.figure(figsize=(12,8))
for (acFunc,hist) in all_convergence_hist.items():
    plt.plot(hist,label = acFunc,marker = 's')
plt.xlabel('Iteration')
plt.ylabel('Objective function')
plt.legend(loc ='best')
plt.savefig('results/rf_gpyopt_acquisitions_comparisons.png',format='png')
plt.show()
#Store the 
all_acquisitions_df = pd.DataFrame(all_convergence_hist)
all_acquisitions_df.to_csv('results/rf_gpyopt_acquisitions_convergence.csv')

EI


INFO:GP:initializing Y
INFO:GP:initializing inference method
INFO:GP:adding kernel and likelihood as parameters


KeyboardInterrupt: ignored

In [0]:
# try a random forest algorithm
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators =100,max_depth = 5,min_samples_leaf = 2,random_state=123,warm_start =True,bootstrap =True )
rf.fit(trainInput,trainOutput)
predictTestOutput = rf.predict(testInput)
#     testMAE = mae(testOutput,predictTestOutput)
testMAPE = mape(testOutput,predictTestOutput)
print(testMAPE)

9.843931196987628


In [0]:
#try a adaboost algorithm
from sklearn.ensemble import AdaBoostRegressor
adaBoost = AdaBoostRegressor(n_estimators = 15,learning_rate=1.0, loss='linear')
adaBoost.fit(trainInput,trainOutput)
predictTestOutput = adaBoost.predict(testInput)
#     testMAE = mae(testOutput,predictTestOutput)
testMAPE = mape(testOutput,predictTestOutput)
print(testMAPE)

10.905318435123439


In [0]:
#bagging regressor
from sklearn.ensemble import BaggingRegressor
bagRegressor = BaggingRegressor(n_estimators=40, max_samples=0.8,max_features=1.0)
bagRegressor.fit(trainInput,trainOutput)
predictTestOutput = bagRegressor.predict(testInput)
#     testMAE = mae(testOutput,predictTestOutput)
testMAPE = mape(testOutput,predictTestOutput)
print(testMAPE)

10.328756543864154


In [0]:
#

In [0]:
#SMAC keras,MLP
def mlp_from_cfg(cfg):
#     cfg = {k : cfg1[k] for k in cfg1 if cfg1[k]}
    cfg = {k : cfg[k] for k in cfg if cfg[k]}
#     print(cfg)
    model = Sequential()
    for index in range(cfg['numLayer']):
        if index == 0:
            print(cfg['count1'])
            model.add(Dense(cfg['count1'],input_dim=5,activation='relu'))#First hidden layer
        else:
            countKey = 'count' + str(index+1)
            print(cfg[countKey])
            model.add(Dense(cfg[countKey],activation='relu'))   
    #Add a output layer layer 
    model.add(Dense(1,activation='linear'))#output layer
    
    model.compile(loss='mape', optimizer='adam', metrics=['mape'])
#     model.summary()
    model.fit(trainInput, trainOutput, epochs=cfg['numEpoch'], batch_size=20, verbose=0, shuffle=False)
    
    predictTestOutput = model.predict(testInput)
#     testMAE = mae(testOutput,predictTestOutput)
    testMAPE = mape(testOutput,predictTestOutput)
#     print('Test MAE: %.5f, Test MAPE: %.5f'%(testMAE,testMAPE))
#     testCoef = coef(testOutput,predictTestOutput)
    return testMAPE
#     return -testCoef


#logger = logging.getLogger("SVMExample")
logging.basicConfig(level=logging.INFO)  # logging.DEBUG for debug output

# Build Configuration Space which defines all parameters and their ranges
cs = ConfigurationSpace()

numLayer = UniformIntegerHyperparameter("numLayer",1,4,default_value=1)
numEpoch = UniformIntegerHyperparameter("numEpoch",10,20,default_value=20)

count1 = UniformIntegerHyperparameter("count1", 3, 4, default_value=3)
count2 = UniformIntegerHyperparameter("count2", 3, 4, default_value=3)
count3 = UniformIntegerHyperparameter("count3", 3, 4, default_value=3)
count4 = UniformIntegerHyperparameter("count4", 3, 4, default_value=3)
cs.add_hyperparameters([numLayer,numEpoch,count1,count2,count3,count4])

#adding dependicies
use_count2 = InCondition(child=count2, parent=numLayer, values=[2])
use_count3 = InCondition(child=count3, parent=numLayer, values=[3])
use_count4 = InCondition(child=count4, parent=numLayer, values=[4])
cs.add_conditions([use_count2,use_count3,use_count4])

# Scenario object
scenario = Scenario({"run_obj": "quality",   # we optimize quality (alternatively runtime)
                     "runcount-limit": 150,  # maximum function evaluations
                     "cs": cs,               # configuration space
                     "deterministic": "true"
                     })
types, bounds = get_types(scenario.cs, scenario.feature_array)
rng =  np.random.RandomState(1)

model = RandomForestWithInstances(types=types,bounds=bounds,\
                                              instance_features=scenario.feature_array,\
                                              seed=rng.randint(MAXINT),\
                                              pca_components=scenario.PCA_DIM,\
                                              log_y=scenario.transform_y in ["LOG", "LOGS"],\
                                              num_trees=scenario.rf_num_trees,\
                                              do_bootstrapping=scenario.rf_do_bootstrapping,\
                                              ratio_features=scenario.rf_ratio_features,\
                                              min_samples_split=scenario.rf_min_samples_split,\
                                              min_samples_leaf=scenario.rf_min_samples_leaf,\
                                              max_depth=scenario.rf_max_depth)

# Example call of the function
# It returns: Status, Cost, Runtime, Additional Infos
def_value = mlp_from_cfg(cs.get_default_configuration())
print("Default Value: %.2f" % (def_value))

# smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
#             tae_runner=svm_from_cfg,acquisition_function =EI(model=model))
smac = SMAC(scenario=scenario, rng=np.random.RandomState(42),
            tae_runner=mlp_from_cfg,acquisition_function =LCB(model=model))
incumbent = smac.optimize()
# print('The final incumbent is:',incumbent)
inc_value = mlp_from_cfg(incumbent)

print("Optimized Value: %.2f" % (inc_value))

In [0]:
#grid search

In [0]:
#random search

In [0]:
#heuristic methods